In [ ]:
pwd

In [ ]:
import pandas as pd

In [ ]:
docking = pd.read_csv('tutorial_chemflow/DockFlow/plants/vmd-rec/SORTED-uniq-lig.csv', delim_whitespace=True)
docking.name = 'docking'

#implicitspinglepoint = pd.read_csv('tutorial_chemflow/ScoreFlow/mmgbsa/vmd-rec/SORTED-uniq-lig.csv', delim_whitespace=True)
#implicitspinglepoint.name = 'implsp'

implicitMD = pd.read_csv('tutorial_chemflow/ScoreFlow/mmgbsa_md/vmd-rec/SORTED-uniq-lig.csv', delim_whitespace=True)
implicitMD.name = 'implicitMD'

explicitMD = pd.read_csv('tutorial_chemflow/ScoreFlow/explicit_mmgbsa/vmd-rec/SORTED-uniq-lig.csv', delim_whitespace=True)
explicitMD.name = 'explicitMD'


In [ ]:
alldfs=[docking, implicitMD, explicitMD]

In [ ]:
# Let's remove the second column with the names of our ligand. They are already specified in the first one!

for i in alldfs:
    i.drop(columns=['LIGAND.1'], inplace=True)

In [ ]:
# Let's rename the column SCORE/RMSD(SMINA-MIN) in simply "SCORE" for comodity in the dataframe docking

docking.rename(
    columns={'SCORE/RMSD(SMINA-MIN)':"SCORE"}
          ,inplace=True)
    

In [ ]:
#Let's make our negative scores positive, in order to do not have problems later on

for i in alldfs:
    i['SCORE'] = i['SCORE']*-1 
    

In [ ]:
# Let's create a new column in each dataframe in which we specify the initial letter of the LIGAND name.
# This is needed to classify our compounds in true active or decoys. 
# We know from the file of CORRESPONDENCE of names which one are the actives and which ones the decoys

for i in alldfs:
    classification = []
    for row in i['LIGAND']:
        if row in ['QVDZMCVSIVITGR-UHFFFAOYSA-N', 'KEDLOQLGBOISAT-UHFFFAOYSA-N', 'MBVYSLWWSSHIRS-UHFFFAOYSA-N', 
                   'ZWXFJUPNUWRODU-UHFFFAOYSA-N', 'PXXJHWLDUBFPOL-UHFFFAOYSA-N', 'KXNPVXPOPUZYGB-XYVMCAHJSA-O',
                  'XXTWZTPVNIYSJZ-XMMPIXPASA-N'] : classification.append(1) # classification for decoys
        else :   classification.append(0) # classification for actives
    i['classification'] = classification
  


In [ ]:
# Let's define the function to generate Receiver operating characteristics (ROC) 
# The number of true actives is 7 and the number of decoys in our dataset is 14. 
# If you work with different datasets remmeber to modify these values!

def roc_tutorial(df, actives=7, decoys=14):
    tp = 0
    fp = 0
    tpr = []
    fpr = []

    for c in df['classification']: 
        if c == 1:
            tp+=1
        elif c == 0:
            fp+=1
        tpr.append(tp/actives)
        fpr.append(fp/decoys)
   
    
    return tpr, fpr
    

In [ ]:
# Let's run the roc_tutorial function for all the tested docking programs.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=100)

lw = 3.5


for i in alldfs:
    auc = metrics.roc_auc_score(i['classification'],  i['SCORE'])
    tpr, fpr = roc_tutorial(i)
    plt.plot(fpr,tpr,label = f"ROC curve {i.name} (AUC = %0.2f)" % auc, alpha = 0.8, lw =lw)


plt.title("Receiver operating characteristic (ROC)\n", fontweight="bold", fontsize =15)

plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")

plt.tick_params(axis='both', labelsize=13)

plt.xlabel("\nFalse Positive Rate",fontsize =14)

plt.ylabel("True Positive Rate\n",fontsize =14)

plt.legend(loc='best')

plt.savefig("ROC-curve-ScoreFlow.png", dpi=300, facecolor='white', transparent=False)
